### Tensorflow 2.0 tutoral based on the IMDB film reviews

Taken from: https://www.tensorflow.org/beta/tutorials/keras/basic_text_classification_with_tfhub

Uses a pretrained text embedding from tfhub

In [57]:
#!pip install tensorflow_hub

In [58]:
#!pip install tensorflow_datasets

In [59]:
#!pip install tfds-nightly

In [60]:
#!pip install tf-nightly

In [61]:
# Didn't need : !pip install protobuf for this pc

In [11]:
# !python --version

Python 3.6.8 :: Anaconda, Inc.


In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

tf.enable_eager_execution()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

W0712 18:22:07.295638  2900 module_wrapper.py:126] From c:\users\alunb\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\util\module_wrapper.py:153: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.



Version:  1.15.0-dev20190712
Eager mode:  True
Hub version:  0.5.0
GPU is NOT AVAILABLE


Download the IMDB dataset

In [2]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

W0712 18:22:17.462471  2900 module_wrapper.py:126] From c:\users\alunb\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\util\module_wrapper.py:153: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

W0712 18:22:17.856939  2900 ag_logging.py:146] Entity <bound method TopLevelFeature.decode_example of FeaturesDict({
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string, encoder=None),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Mangled names are not yet supported by AutoGraph


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string, encoder=None),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Mangled names are not yet supported by AutoGraph


W0712 18:22:17.991343  2900 ag_logging.py:146] Entity <bound method TopLevelFeature.decode_example of FeaturesDict({
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string, encoder=None),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Mangled names are not yet supported by AutoGraph


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string, encoder=None),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Mangled names are not yet supported by AutoGraph


W0712 18:22:18.067258  2900 ag_logging.py:146] Entity <bound method TopLevelFeature.decode_example of FeaturesDict({
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string, encoder=None),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Mangled names are not yet supported by AutoGraph


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string, encoder=None),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Mangled names are not yet supported by AutoGraph


In [3]:
train_data

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [5]:
train_examples_batch

<tf.Tensor: id=224, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [6]:
train_labels_batch

<tf.Tensor: id=225, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)>

Choose a text embedding from a previously trained model, here use a pre-trained text embedding model from TensorFlow Hub called google/tf2-preview/gnews-swivel-20dim/1.

There are three other pre-trained models to test for the sake of this tutorial:

- google/tf2-preview/gnews-swivel-20dim-with-oov/1 - same as google/tf2-preview/gnews-swivel-20dim/1, but with 2.5% vocabulary converted to OOV buckets. This can help if vocabulary of the task and vocabulary of the model don't fully overlap.
- google/tf2-preview/nnlm-en-dim50/1 - A much larger model with ~1M vocabulary size and 50 dimensions.
- google/tf2-preview/nnlm-en-dim128/1 - Even larger model with ~1M vocabulary size and 128 dimensions.

Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences, and try it out on a couple of input examples. Note that no matter the length of the input text, the output shape of the embeddings is: 

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=404, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

Build the model

- Layer 1: TF hub layer with pre-trained, 20D word embedding
- Layer 2: 16 node ReLU layer
- Layer 3: 1 node sigmoid (binary classificaiton)
(Remember that the sigmoid function = $1/(1+e^{-x})$)

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


Configure optimizer and loss funciton

In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Train the model on mini batches of 512

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20


W0712 18:33:58.400442  2900 deprecation.py:323] From c:\users\alunb\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1251: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0712 18:33:58.636646  2900 deprecation.py:323] From c:\users\alunb\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\keras\optimizer_v2\optimizer_v2.py:454: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


30/30 [==============================].7201 - acc: 0.632 - 3s 2s/step - loss: 0.7334 - acc: 0.608 - 3s 1s/step - loss: 0.7133 - acc: 0.623 - 3s 852ms/step - loss: 0.7143 - acc: 0.61 - 4s 722ms/step - loss: 0.7062 - acc: 0.62 - 4s 636ms/step - loss: 0.7038 - acc: 0.62 - 4s 573ms/step - loss: 0.6989 - acc: 0.62 - 4s 528ms/step - loss: 0.6948 - acc: 0.62 - 4s 493ms/step - loss: 0.6929 - acc: 0.62 - 5s 461ms/step - loss: 0.6861 - acc: 0.63 - 5s 431ms/step - loss: 0.6824 - acc: 0.63 - 5s 407ms/step - loss: 0.6749 - acc: 0.63 - 5s 387ms/step - loss: 0.6697 - acc: 0.64 - 5s 369ms/step - loss: 0.6671 - acc: 0.64 - 5s 354ms/step - loss: 0.6661 - acc: 0.64 - 5s 341ms/step - loss: 0.6623 - acc: 0.64 - 6s 328ms/step - loss: 0.6588 - acc: 0.64 - 6s 318ms/step - loss: 0.6564 - acc: 0.64 - 6s 308ms/step - loss: 0.6520 - acc: 0.64 - 6s 300ms/step - loss: 0.6502 - acc: 0.65 - 6s 292ms/step - loss: 0.6482 - acc: 0.65 - 6s 286ms/step - loss: 0.6453 - acc: 0.65 - 6s 279ms/step - loss: 0.6452 - acc: 0.65 -

30/30 [==============================] - ETA: 49s - loss: 0.4073 - acc: 0.81 - ETA: 26s - loss: 0.3933 - acc: 0.83 - ETA: 19s - loss: 0.3851 - acc: 0.83 - ETA: 15s - loss: 0.3885 - acc: 0.83 - ETA: 12s - loss: 0.3887 - acc: 0.83 - ETA: 11s - loss: 0.3899 - acc: 0.83 - ETA: 9s - loss: 0.3917 - acc: 0.8309 - ETA: 8s - loss: 0.3904 - acc: 0.833 - ETA: 7s - loss: 0.3922 - acc: 0.832 - ETA: 7s - loss: 0.3894 - acc: 0.835 - ETA: 6s - loss: 0.3880 - acc: 0.836 - ETA: 5s - loss: 0.3857 - acc: 0.838 - ETA: 5s - loss: 0.3849 - acc: 0.838 - ETA: 4s - loss: 0.3845 - acc: 0.840 - ETA: 4s - loss: 0.3846 - acc: 0.840 - ETA: 3s - loss: 0.3825 - acc: 0.841 - ETA: 3s - loss: 0.3818 - acc: 0.841 - ETA: 3s - loss: 0.3801 - acc: 0.842 - ETA: 2s - loss: 0.3797 - acc: 0.843 - ETA: 2s - loss: 0.3804 - acc: 0.842 - ETA: 2s - loss: 0.3806 - acc: 0.843 - ETA: 1s - loss: 0.3797 - acc: 0.843 - ETA: 1s - loss: 0.3804 - acc: 0.842 - ETA: 1s - loss: 0.3795 - acc: 0.842 - ETA: 1s - loss: 0.3801 - acc: 0.842 - ETA: 0s 

30/30 [==============================] - ETA: 43s - loss: 0.2729 - acc: 0.89 - ETA: 24s - loss: 0.2549 - acc: 0.90 - ETA: 17s - loss: 0.2453 - acc: 0.90 - ETA: 13s - loss: 0.2449 - acc: 0.90 - ETA: 11s - loss: 0.2455 - acc: 0.90 - ETA: 10s - loss: 0.2476 - acc: 0.90 - ETA: 9s - loss: 0.2492 - acc: 0.9054 - ETA: 8s - loss: 0.2467 - acc: 0.907 - ETA: 7s - loss: 0.2474 - acc: 0.905 - ETA: 6s - loss: 0.2454 - acc: 0.906 - ETA: 5s - loss: 0.2432 - acc: 0.907 - ETA: 5s - loss: 0.2418 - acc: 0.909 - ETA: 4s - loss: 0.2415 - acc: 0.909 - ETA: 4s - loss: 0.2413 - acc: 0.909 - ETA: 4s - loss: 0.2415 - acc: 0.910 - ETA: 3s - loss: 0.2398 - acc: 0.910 - ETA: 3s - loss: 0.2391 - acc: 0.910 - ETA: 2s - loss: 0.2375 - acc: 0.911 - ETA: 2s - loss: 0.2388 - acc: 0.910 - ETA: 2s - loss: 0.2395 - acc: 0.910 - ETA: 2s - loss: 0.2401 - acc: 0.910 - ETA: 1s - loss: 0.2396 - acc: 0.910 - ETA: 1s - loss: 0.2407 - acc: 0.910 - ETA: 1s - loss: 0.2396 - acc: 0.910 - ETA: 1s - loss: 0.2401 - acc: 0.910 - ETA: 0s 

30/30 [==============================] - ETA: 48s - loss: 0.1760 - acc: 0.95 - ETA: 26s - loss: 0.1645 - acc: 0.94 - ETA: 19s - loss: 0.1584 - acc: 0.94 - ETA: 15s - loss: 0.1559 - acc: 0.94 - ETA: 12s - loss: 0.1564 - acc: 0.94 - ETA: 10s - loss: 0.1585 - acc: 0.94 - ETA: 9s - loss: 0.1603 - acc: 0.9436 - ETA: 8s - loss: 0.1585 - acc: 0.945 - ETA: 7s - loss: 0.1590 - acc: 0.944 - ETA: 7s - loss: 0.1577 - acc: 0.944 - ETA: 6s - loss: 0.1561 - acc: 0.946 - ETA: 5s - loss: 0.1551 - acc: 0.946 - ETA: 5s - loss: 0.1549 - acc: 0.946 - ETA: 4s - loss: 0.1551 - acc: 0.946 - ETA: 4s - loss: 0.1555 - acc: 0.947 - ETA: 3s - loss: 0.1545 - acc: 0.946 - ETA: 3s - loss: 0.1540 - acc: 0.947 - ETA: 3s - loss: 0.1527 - acc: 0.947 - ETA: 2s - loss: 0.1549 - acc: 0.947 - ETA: 2s - loss: 0.1556 - acc: 0.947 - ETA: 2s - loss: 0.1568 - acc: 0.946 - ETA: 1s - loss: 0.1567 - acc: 0.947 - ETA: 1s - loss: 0.1581 - acc: 0.946 - ETA: 1s - loss: 0.1570 - acc: 0.947 - ETA: 1s - loss: 0.1574 - acc: 0.947 - ETA: 0s 

Evaluate the model

In [11]:
results = model.evaluate(test_data.batch(512), verbose=0)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.323
acc: 0.865


Let's calculate th output for a random example

In [56]:
example = next(iter(test_data.shuffle(10000).batch(1)))
print(example[0][0].numpy())
print('\nActual: {}, Predicted: {}'.format(example[1][0].numpy(), model.predict(example)))

b'I was really surprised to see that unlike most documentaries, this was written, directed and produced by a film critic-- Richard Schickel. Most of the times I know of where film critics had major involvement in films, the films turns out to be bombs (Rex Reed starring in "Myra Breckenridge" and Roger Ebert writing "Beyond the Valley of the Dolls" are prime examples). However, in this case, the critic\'s powers are used for good and not evil--and the results are better (though this isn\'t saying much, as the films I just mentioned are among the worst films ever made).<br /><br />As for the documentary, it\'s narrated by Clint Eastwood (a pretty good choice) and manages to discuss his long career--from his silent days until his death in the early 1960s. The only negatives, and they are slight, are that the film is awfully short (as are most film documentaries) and there is very little about Gary Cooper as a human being--you really don\'t learn all that much about his life. However, as 